# SO, SO2, PPD: comunicação e sincronização de *threads*

Hélio - DC/UFSCar - 2023

# Sincronização de *threads*

*Threads* compartilham o espaço de endereçamento do processo ao qual estão associadas. Isso facilita as comunicações entre elas, uma vez que quaisquer estruturas manipuladas de maneira particionada podem ser posicionadas em áreas de memória comuns, acessíveis por todas as *threads*.

Por outro lado, a consistência dos dados compartilhados pode depender de certos acessos ocorrerem de **maneira exclusiva** e é claro que essa sincronização não seria eficiente se fosse implementada com algum tipo de ***busy waiting***, salvo em bloqueios de curtíssima duração.

```
while(recurso_ocupado())
  fica_aqui_à_toa_gastando_fatia_de_tempo_até_outra_tarefa_liberá-lo();
```

Assim, a ideia é ter mecanismos de sincronização e exclusão mútua que possibilitem que as tarefas sejam automaticamente suspensas quando condições necessárias não são satisfeitas.

Desta forma, a tarefa que aguarda uma condição pode ser tirada de execução e outra tarefa pode ser selecionada para ocupar o processador que se tornou disponível agora.

O primeiro passo para isso é ter uma biblioteca padronizada que ofereça esses serviços. No cado do uso de *threads* com a biblioteca ***pthreads***, há uma série de estruturas de sincronização, como pode ser visto em [pthread.h](https://man7.org/linux/man-pages/man0/pthread.h.0p.html).

É útil para a aplicação usar bibliotecas padronizadas, já que isso aumenta a portabilidade dos programas.

* No caso específico do sistema Linux, por razões de eficiência, as implementações dessas bibliotecas procuram fazer todas as operações possíveis no "espaço de usuário" (*user space*), evitando o custo dos mecanismos de chamada de sistema. Isto é importante, já que a eficiência dos mecanismos de sincronização é fundamental para o bom desempenho das aplicações.

* Quando o apoio do SO (Linux) é necessário para as sincronizações, a chamada [futex](https://man7.org/linux/man-pages/man2/futex.2.html) pode ser usada, por exemplo, tanto para bloquear (***FUTEX_WAIT***), quanto para acordar (***FUTEX_WAKE***) tarefas.

Quando vai bloquear uma tarefa, o SO primeiro ajusta e salva informações de contexto da tarefa que emite a chamada. O descritor desta tarefa é então "desligado" da lista de tarefas prontas e inserido na fila das tarefas que aguardam que essa condição seja satisfeita. Tipicamente, a tarefa é inserida na fila de tarefas associada ao semáforo ou mutex em questão (ou futex, no Linux). O SO segue selecionando outra tarefa pronta para execução e restaurando seu contexto para que ela volte a ser executada.

Pensando que estamos estudando a lógica dos mecanismos de sincronização e que queremos criar programas portáteis, sigamos usando as APIs POSIX e os mecanismos providos para isto!



# Variáveis compartilhadas

Em muitos casos, a sincronização de tarefas é necessária apenas para a atualização de alguma variável.

Se esse ajuste é apenas de escrita, ou seja, é preciso apenas sobrepor o valor atual desta variável, trata-se de uma operação rápida, que consome apenas 1 instrução do processador, e não há risco de gerar-se valores inconsistentes.

Já se a atualização depende do valor atual desta variável, então é preciso ler o valor atual, aplicar a modificação e salvar o novo valor. É aí que está o problema: e se uma *thread* **lê** o valor, ajusta sua cópia e, antes de salvar o valor atualizado, o valor original for **lido** por outra *thread*, que irá também atualizá-lo e salvá-lo?

O valor resultante vai ser aquele que for salvo por último, sem refletir a mudança que foi feita por uma das *threads* neste caso.


# Atualizando variável global por múltiplas *threads*

Mas, qual é o risco efetivo de isto ocorrer?

Considere 8 *threads* que querem alterar o valor de uma variável compartilhada, sendo executadas em um computador com 8 *cores*. Teoricamente, se não há outras tarefas prontas para execução, todas podem estar executando essa operação **ao mesmo tempo**, já que há um *core* para cada uma.

Num outro cenário, é possível ter mais *threads* do que processadores, de forma que o SO vai ficar alternando o uso do(s) processsador(es) entre elas.

Como cada *thread* vai receber **fatias de tempo** de uso do processador, é possível que o instante de troca de contexto ocorra exatamente entre a instrução de consulta e a atualização da variável compartilhada.

Vejamos um exemplo de código em que muitas *threads* vão tentar incremenar um contador muitas vezes, sem exclusão mútua.

Executem o programa várias vezes e observem o valor resultante, que deveria ser igual ao número de *threads* vezes o número de iterações.



In [ ]:
%%writefile global.c

/* Baseado em
   https://stackoverflow.com/questions/2353371/how-to-do-an-atomic-increment-and-fetch-in-c
   https://en.cppreference.com/w/c/language/atomic
 */

#include <pthread.h>
#include <stdio.h>

// #define  NUM_THREADS   1000
// #define  NUM_ITERS     1000
#define  NUM_THREADS   100
#define  NUM_ITERS     10000

int global = 0;

void*
threadFun (void *arg)
{
  for (int i = 0; i < NUM_ITERS; ++i)
    global++;

  return NULL;
}

int
main(void)
{
  int i;
  pthread_t threads[NUM_THREADS];

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_create(&threads[i], NULL, threadFun, NULL);

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_join(threads[i], NULL);

  printf("global: %d\n",global);

  return 0;
}


Writing global.c


In [ ]:
! if [ ! global -nt global.c ]; then gcc -Wall global.c -o global -pthread; fi
! time ./global

global: 969205

real	0m0.012s
user	0m0.003s
sys	0m0.006s


Dificilmente gerou 1.000.000, né?!

# Sincronizando com *mutexes*

Vejamos agora como usar um ***mutex*** para a sincronização neste caso.

A região crítica é o trecho de código que tem que ser executado com exclusão mútua. Neste problema, é o trecho em que uma *thread* incrementa o contador a partir de seu valor atual.

In [ ]:
%%writefile global-mtx.c

#include <pthread.h>
#include <stdio.h>

// #define  NUM_THREADS   1000
// #define  NUM_ITERS     1000
#define  NUM_THREADS   100
#define  NUM_ITERS     10000

int global = 0;

pthread_mutex_t _mutex;

void*
threadFun (void *arg)
{
  for (int i = 0; i < NUM_ITERS; ++i) {
    pthread_mutex_lock(&_mutex);
      global++;                        // seção crítica
    pthread_mutex_unlock(&_mutex);
  }
  return NULL;
}

int
main(void)
{
  int i;
  pthread_t threads[NUM_THREADS];

  pthread_mutex_init(&_mutex, NULL);

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_create(&threads[i], NULL, threadFun, NULL);

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_join(threads[i], NULL);

  printf("global: %d\n",global);

  pthread_mutex_destroy (&_mutex);

  return 0;
}


Writing global-mtx.c


In [ ]:
! if [ ! global-mtx -nt global-mtx.c ]; then gcc -Wall global-mtx.c -o global-mtx -pthread; fi
! time ./global-mtx

global: 1000000

real	0m0.066s
user	0m0.045s
sys	0m0.032s


Com a execução do código acima, vê-se que o valor final do contador está correto.

Por outro lado, a execução foi significativamente mais lenta.

# Sincronizando com *spins*

Além dos *mutexes*, é possível usar ***spin*** para a sincronização. Como a operação de atualização do contado requer um bloqueio de curtíssima duração, parece que não seria ruim usar ***busy waiting*** neste caso.

Isso é o que faz o mecanismo de *spin*, sem risco de haver a suspensão da tarefa corrente se houver uma outra tarefa em sua seção crítica, executando com exclusão mútua.

Vejam que o código é praticamente igual ao que usa *mutex*!

In [ ]:
%%writefile global-spin.c

#include <pthread.h>
#include <stdio.h>

// #define  NUM_THREADS   1000
// #define  NUM_ITERS     1000
#define  NUM_THREADS   100
#define  NUM_ITERS     10000

int global = 0;

// pthread_mutex_t _mutex;
pthread_spinlock_t _slock;

void*
threadFun (void *arg)
{
  for (int i = 0; i < NUM_ITERS; ++i) {
    // pthread_mutex_lock(&_mutex);
	  pthread_spin_lock(&_slock);
      global++;
    // pthread_mutex_unlock(&_mutex);
	  pthread_spin_unlock(&_slock);
  }
  return NULL;
}

int
main(void)
{
  int i;
  pthread_t threads[NUM_THREADS];

  // pthread_mutex_init(&_mutex, NULL);
	pthread_spin_init(&_slock, PTHREAD_PROCESS_PRIVATE);

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_create(&threads[i], NULL, threadFun, NULL);

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_join(threads[i], NULL);

  printf("global: %d\n",global);

  // pthread_mutex_destroy (&_mutex);
	pthread_spin_destroy (&_slock);

  return 0;
}


Writing global-spin.c


In [ ]:
! if [ ! global-spin -nt global-spin.c ]; then gcc -Wall global-spin.c -o global-spin -pthread; fi
! time ./global-spin

global: 1000000

real	0m0.025s
user	0m0.011s
sys	0m0.004s


# Instruções de acesso à memória

Vejamos o código C a seguir, que ilustra a atualização do conteúdo de uma variável,

```
int
main()
{
  int val;
  val = 3;
  val = val +2;

  return 0;
}
```
cujo código *assembly* pode ser visto a seguir.


```
TDump of assembler code for function main:
   0x0000000100003f90  <+0>:	push   %rbp
   0x0000000100003f91  <+1>:	mov    %rsp,%rbp
   0x0000000100003f94  <+4>:	xor    %eax,%eax
   0x0000000100003f96  <+6>:	movl   $0x0,-0x4(%rbp)
   0x0000000100003f9d <+13>:	movl   $0x3,-0xc(%rbp)
   0x0000000100003fa4 <+20>:	mov    -0xc(%rbp),%ecx
   0x0000000100003fa7 <+23>:	add    $0x2,%ecx
   0x0000000100003faa <+26>:	mov    %ecx,-0xc(%rbp)
   0x0000000100003fad <+29>:	pop    %rbp
   0x0000000100003fae <+30>:	ret    
End of assembler dump.
```

Com relação à variável e ao valor de interesse:

* **val** foi alocada na posição **topo da pilha** (%rbp) -12(0xc)
* o conteúdo desta posição de memória é copiado para um registrador (ecx)
* O valor a ser somato é adicionado diretamente ao conteúdo do registrador (ecx)
* o valor resultante no registrador é copiado para a posição de memória reservada para a variável (rbp -0xc).

Como se vê, para alterar o conteúdo de uma posição de memória, seu valor foi copiado para um registrador (ecx), onde recebeu o incremento, e depois foi copiado de volta para a posição de memória.

<br>

# Incremento atômico

Não se trata de enriquecimento de Urânio ou coisa assim :-/ O uso do termo **operação atômica** em computação, serve para identificar um bloco de código indivisível, ou seja, cujas instruções devem ser executadas em totalidade antes que o processador seja alocado a outra tarfa.

Felizmente, há nos processadores instruções que conseguem ler e alterar o valor de uma posição de memória em um único ciclo, ou ao menos impedir o acesso à memória entre uma operação de leitura e escrita na mesma posição.

Tipicamente, esse recurso é chamado de [Fetch-and-add](https://en.wikipedia.org/wiki/Fetch-and-add), mas há outras operações atômicas possíveis também.

Há ainda operações do tipo [Test-and-set](https://en.wikipedia.org/wiki/Test-and-set), que também são úteis para realizar uma consulta e alteração do valor de uma posição de memória de forma atômica.

Em termos de uso desses mecanismos na programação em alto nível, há recursos na linguagem C/C++, por exemplo que traduzem uma especificação lógica desta forma em instruções do processador usado na geração de código.

https://en.cppreference.com/w/c/atomic/atomic_fetch_add

https://en.wikipedia.org/wiki/Compare-and-swap


## C/C++ atomic bultins


Em C/C++ há ainda uma série de construções que permitem realizar acessos atômicos a variáveis sem bloqueio.

Uma lista dessas construções pode ser vista em https://gcc.gnu.org/onlinedocs/gcc/_005f_005fatomic-Builtins.html

Isso inclui, por exemplo:

* Built-in Function: type __atomic_fetch_add (type *ptr, type val, int memorder)
* Built-in Function: type __atomic_fetch_sub (type *ptr, type val, int memorder)
* Built-in Function: type __atomic_fetch_and (type *ptr, type val, int memorder)
* Built-in Function: type __atomic_fetch_xor (type *ptr, type val, int memorder)
* Built-in Function: type __atomic_fetch_or (type *ptr, type val, int memorder)
* Built-in Function: type __atomic_fetch_nand (type *ptr, type val, int memorder)

No caso da manipulação que estamos fazendo no programa *multithreaded* exemplo, podemos usar ***__atomic_fetch_add***.

```
int global = 0;

    __atomic_fetch_add(&global, 1, __ATOMIC_SEQ_CST);
```

Cabe à implementação do compilador tratar isso, que em geral vai ser traduzido para alguma instrução que permite a manipulação desejada de forma atômica.



# C/C++: _Atomic

Em C/C++, há algumas formas para fazer com que algumas operações de atualização de conteúdos de posição de memória sejam feitas com o suporte de instruções do processador para realizar isso de forma **atômica**.

É importante observar, contudo, que a atomicidade é obtida ***sem o uso de bloqueios***, mas explorando instruções específicas do processador para o qual o código é gerado na compilação.


## _Atomic

Objetos do tipo *atomic* são os únicos **livres de condição de corrida** (***data races***), e podem ser modificados simultaneamente por mais de uma *threads*.

```
Ex:

_Atomic int contador;
```


https://en.cppreference.com/w/c/language/atomic

https://lumian2015.github.io/lockFreeProgramming/c11-features-in-currency.html

```
Syntax

_Atomic ( type-name )   (1) 	(since C11)
_Atomic type-name       (2) 	(since C11)

1) Use as a type specifier; this designates a new atomic type
2) Use as a type qualifier; this designates the atomic version of type-name.
   In this role, it may be mixed with const, volatile, and restrict), although
   unlike other qualifiers, the atomic version of type-name may have a
   different size, alignment, and object representation.

The header <stdatomic.h> defines 37 convenience macros, from atomic_bool to
atomic_uintmax_t, which simplify the use of this keyword with built-in and
library types.

_Atomic const int * p1;  // p is a pointer to an atomic const int
const atomic_int * p2;   // same
const _Atomic(int) * p3; // same
```

```
Objects of atomic types are the only objects that are free from data races,
that is, they may be modified by two threads concurrently or modified by one
and read by another.

Each atomic object has its own associated modification order, which is a total
order of modifications made to that object. If, from some thread's point of
view, modification A of some atomic M happens-before modification B of the same
atomic M, then in the modification order of M, A occurs before B.

Note that although each atomic object has its own modification order, there is
no single total order; different threads may observe modifications to different
atomic objects in different orders.

There are four coherences that are guaranteed for all atomic operations:

* write-write coherence: If an operation A that modifies an atomic object M
  happens-before an operation B that modifies M, then A appears earlier than B
  in the modification order of M.

* read-read coherence: If a value computation A of an atomic object M happens
  before a value computation B of M, and A takes its value from a side effect X
  on M, then the value computed by B is either the value stored by X or is the
  value stored by a side effect Y on M, where Y appears later than X in the
  modification order of M.

* read-write coherence: If a value computation A of an atomic object M
  happens-before an operation B on M, then A takes its value from a side effect
  X on M, where X appears before B in the modification order of M.

* write-read coherence: If a side effect X on an atomic object M happens-before
  a value computation B of M, then the evaluation B takes its value from X or
  from a side effect Y that appears after X in the modification order of M.
```



Uma observação em relação à compilação de programas que usam o recurso __Atomic no gcc é inclusão de um arquivo de definições específico (<stdatomic.h>) e incluir explicitamente a biblioteca ***libatomic*** na geração de código.

```Please be sure to include the header `#include<stdatomic.h>" when you are us`ng the atomic type in your program.```

A ligação do código é feita com o parâmetro **-latomic**, como ilustrado a seguir:

```
$ gcc prog.c  ... .... -latomic
```

In [ ]:
%%writefile atomic.c

/* https://stackoverflow.com/questions/2353371/how-to-do-an-atomic-increment-and-fetch-in-c */

#include <pthread.h>
#include <stdlib.h>
#include <stdio.h>
#include <stdatomic.h>

#define  NUM_THREADS 100
#define  NUM_ITERS   10000

_Atomic int atomic_global = 0;
int global = 0;
int outra_global =0;

void*
main_thread(void *arg)
{
  int i;

  for (i = 0; i < NUM_ITERS; ++i) {

    // usando operação "instrinsic", que mapeia diretamente para instrução do processador
    __atomic_fetch_add(&global, 1, __ATOMIC_SEQ_CST);

    // usando variável declarada como __Atomic
    atomic_global++;

    // Esta forma é sujeita a falhas...
    outra_global++;
  }
  return NULL;
}

int
main(void)
{
  int i;
  pthread_t threads[NUM_THREADS];

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_create(&threads[i], NULL, main_thread, NULL);

  for (i = 0; i < NUM_THREADS; ++i)
    pthread_join(threads[i], NULL);

  printf("global: %d\n", global);
  printf("_Atomic global: %d\n", atomic_global);
  printf("outra global: %d\n", outra_global);

  return 0;
}


Writing atomic.c


In [ ]:
! if [ ! atomic -nt atomic.c ]; then gcc -Wall atomic.c -o atomic -pthread; fi
! time ./atomic

global: 1000000
_Atomic global: 1000000
outra global: 873788

real	0m0.028s
user	0m0.031s
sys	0m0.006s


# Instruções x64

Vejamos o código gerado pelo compilador C para os recursos de operações atômicas, a declaração ***_Atomic*** e um exemplo de primitica *intrinsic* ***__atomic_fetch_add()***.



In [ ]:
%%writefile at.c

_Atomic int atomic = 0;

int
main()
{
  atomic = atomic +2;

  return 0;
}

Writing at.c


```
(gdb) disass
Dump of assembler code for function main:
   0x000055ea3f8c466a <+0>:	push   %rbp
   0x000055ea3f8c466b <+1>:	mov    %rsp,%rbp
   0x000055ea3f8c466e <+4>:	sub    $0x10,%rsp
=> 0x000055ea3f8c4672 <+8>:	mov    %fs:0x28,%rax
   0x000055ea3f8c467b <+17>:	mov    %rax,-0x8(%rbp)
   0x000055ea3f8c467f <+21>:	xor    %eax,%eax
   0x000055ea3f8c4681 <+23>:	mov    0x20098d(%rip),%eax        # 0x55ea3fac5014 <atomic>
   0x000055ea3f8c4687 <+29>:	mov    %eax,-0x10(%rbp)
   0x000055ea3f8c468a <+32>:	mov    -0x10(%rbp),%eax
   0x000055ea3f8c468d <+35>:	add    $0x2,%eax
   0x000055ea3f8c4690 <+38>:	mov    %eax,-0xc(%rbp)
   0x000055ea3f8c4693 <+41>:	mov    -0xc(%rbp),%eax
   0x000055ea3f8c4696 <+44>:	mov    %eax,0x200978(%rip)        # 0x55ea3fac5014 <atomic>
   0x000055ea3f8c469c <+50>:	mfence
   0x000055ea3f8c469f <+53>:	mov    $0x0,%eax
   0x000055ea3f8c46a4 <+58>:	mov    -0x8(%rbp),%rdx
   0x000055ea3f8c46a8 <+62>:	xor    %fs:0x28,%rdx
   0x000055ea3f8c46b1 <+71>:	je     0x55ea3f8c46b8 <main+78>
   0x000055ea3f8c46b3 <+73>:	callq  0x55ea3f8c4540 <__stack_chk_fail@plt>
   0x000055ea3f8c46b8 <+78>:	leaveq
   ```

Segundo as especificaçẽs do processador X86_64, a instrução **mfence** força a consistência das operações de acesso à memória (*load & store*) anteriores à invocação desta instrução.

*Performs a serializing operation on all load-from-memory and store-to-memory in`tructions that were issued prior the MFENCE instruction.*
*This serializing operation guarantees that every load and store instruction that precedes the MFENCE instruction in program order becomes globally visible before any load or store instruction that follows the MFENCE instruction.*
*The MFENCE instruction is ordered with respect to all load and store instructions, other MFENCE instructions, any LFENCE and SFENCE instructions, and any serializing instructions (such as the CPUID instruction).*
*MFENCE does not serialize the instruction stream.*

https://www.felixcloutier.com/x86/mfence



In [ ]:
! if $( ! apt list gdb | grep "installed" &> /dev/null ) ; then apt install gdb ; fi
! if [ ! at -nt at.c ]; then gcc -Wall -g at.c -o at ; fi
# ! objdump -d -j .text ./at
# ! gdb -batch -ex "disassemble/rs main" ./at
! echo && objdump -D ./at | grep -A25 main.:




0000000000001149 <main>:
    1149:	f3 0f 1e fa          	endbr64 
    114d:	55                   	push   %rbp
    114e:	48 89 e5             	mov    %rsp,%rbp
    1151:	48 83 ec 10          	sub    $0x10,%rsp
    1155:	64 48 8b 04 25 28 00 	mov    %fs:0x28,%rax
    115c:	00 00 
    115e:	48 89 45 f8          	mov    %rax,-0x8(%rbp)
    1162:	31 c0                	xor    %eax,%eax
    1164:	8b 05 aa 2e 00 00    	mov    0x2eaa(%rip),%eax        # 4014 <atomic>
    116a:	89 45 f0             	mov    %eax,-0x10(%rbp)
    116d:	8b 45 f0             	mov    -0x10(%rbp),%eax
    1170:	83 c0 02             	add    $0x2,%eax
    1173:	89 45 f4             	mov    %eax,-0xc(%rbp)
    1176:	8b 45 f4             	mov    -0xc(%rbp),%eax
    1179:	87 05 95 2e 00 00    	xchg   %eax,0x2e95(%rip)        # 4014 <atomic>
    117f:	b8 00 00 00 00       	mov    $0x0,%eax
    1184:	48 8b 55 f8          	mov    -0x8(%rbp),%rdx
    1188:	64 48 2b 14 25 28 00 	sub    %fs:0x28,%rdx
    118f:	00 00 
    1191:

[Intel® 64 and IA-32 Architectures Software Developer's  Manual Combined Volumes 3A, 3B, 3C, and 3D: System Programming Guide](https://cdrdv2.intel.com/v1/dl/getContent/671447)

...<br>
**XCHG** Exchange Register/Memory With Register<br>
...<br>
*Exchanges the contents of the destination (first) and source (second) operands. The operands can be two general-purpose registers or a register and a memory location. If a memory operand is referenced, the processor’s locking
protocol is automatically implemented for the duration of the exchange operation, regardless of the presence or absence of the LOCK prefix or of the value of the IOPL.*
<br>
*This instruction is useful for implementing semaphores or similar data structures for process synchronization. (See “Bus Locking” in Chapter 9 of the Intel® 64 and IA-32 Architectures Software Developer’s Manual, Volume 3A, for
more information on bus locking.)*
<br>
...<br>
**LOCK** Assert LOCK# Signal Prefix<br>
...<br>
Causes the processor’s LOCK# signal to be asserted during execution of the accompanying instruction (turns the instruction into an atomic instruction).
<br>
In a multiprocessor environment, the LOCK# signal ensures that the
processor has exclusive use of any shared memory while the signal is asserted.
<br>
The LOCK prefix can be prepended only to the following instructions and only to those forms of the instructions where the destination operand is a memory operand: ADD, ADC, AND, BTC, BTR, BTS, CMPXCHG, CMPXCH8B, CMPXCHG16B, DEC, INC, NEG, NOT, OR, SBB, SUB, XOR, XADD, and XCHG.
<br>
...<br>
*The XCHG instruction always asserts the LOCK# signal regardless of the presence or absence of the LOCK prefix.*

*The LOCK prefix is typically used with the BTS instruction to **perform a read-modify-write operation** on a memory location in shared memory environment.*
...<br>
...<br>



In [ ]:
%%writefile faa.c

int global;

int
main()
{
  __atomic_fetch_add(&global, 1, __ATOMIC_SEQ_CST);

  return 0;
}


Writing faa.c


In [ ]:
! if $( ! apt list gdb | grep "installed" &> /dev/null ) ; then apt install gdb ; fi
! if [ ! faa -nt faa.c ]; then gcc -Wall -g faa.c -o faa ; fi
# ! objdump -d -j .text ./faa
! gdb -batch -ex "disassemble/rs main" ./faa
# ! echo && objdump -D ./faa | grep -A9 main.:



Dump of assembler code for function main:
faa.c:
6	{
   0x0000000000001129 <+0>:	f3 0f 1e fa	endbr64 
   0x000000000000112d <+4>:	55	push   %rbp
   0x000000000000112e <+5>:	48 89 e5	mov    %rsp,%rbp

7	  __atomic_fetch_add(&global, 1, __ATOMIC_SEQ_CST);
   0x0000000000001131 <+8>:	f0 83 05 db 2e 00 00 01	lock addl $0x1,0x2edb(%rip)        # 0x4014 <global>

8	
9	  return 0;
   0x0000000000001139 <+16>:	b8 00 00 00 00	mov    $0x0,%eax

10	}
   0x000000000000113e <+21>:	5d	pop    %rbp
   0x000000000000113f <+22>:	c3	ret    
End of assembler dump.


Examinando o código *assembly* mostrado anteriormente, vê-se o uso do **prefixo** ***lock*** antes da instrução ***addl***. Segundo o [manual dos processadores X64 Intel](https://www.intel.com/content/dam/www/public/us/en/documents/manuals/64-ia-32-architectures-software-developer-instruction-set-reference-manual-325383.pdf), isso torna a instrução atômica, garantindo que o processador tenha acesso exclusivo ao uso de qualquer memória compartilhada para a operação seguinte.

```
Causes the processor’s LOCK# signal to be asserted during execution of the
accompanying instruction (turns the instruction into an atomic instruction).
In a multiprocessor environment, the LOCK# signal ensures that the processor
has exclusive use of any shared memory while the signal is asserted.
...
ADD
DescriptionAdds the destination operand (first operand) and the source operand
(second operand) and then stores the result in the destination operand.
...
This instruction can be used with a LOCK prefix to allow the instruction to be
executed atomically
```
Pronto, está feita a operação se ajuste do valor de um contador, a partir de seu valor atual, de foma atômica!

Como se vê, isso pode ser bem mais rápido, e igualmente eficiente, em casos de bloqueio para ajuste de variável compartilhada!

Bons estudos e bons programas,

Hélio